In [33]:
import pickle
import numpy as np
import tensorflow as tf
import nltk
import random
import json
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
print(tf.__version__)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

2.15.0


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Symtom Model

<h1>Find mohamed_tokenizer.pickle ,mohamed_tokenizer.pickle
in symtom folder to load both of them </h1>

In [ ]:
all_symp = ['Acne', 'Arthritis', 'Bronchial Asthma', 'Cervical spondylosis',
       'Chicken pox', 'Common Cold', 'Dengue', 'Dimorphic Hemorrhoids',
       'Fungal infection', 'Hypertension', 'Impetigo', 'Jaundice',
       'Malaria', 'Migraine', 'Pneumonia', 'Psoriasis', 'Typhoid',
       'Varicose Veins', 'allergy', 'diabetes', 'drug reaction',
       'gastroesophageal reflux disease', 'peptic ulcer disease',
       'urinary tract infection']

try:
    with open('/content/mohamed_tokenizer.pickle', 'rb') as t: ## upload mohamed_tokenizer.pickle and put the path here
        tokenizer = pickle.load(t)
except Exception as e:
    print("Error loading tokenizer:", e)
model_symp = tf.keras.models.load_model('/content/mohamed_symptom.h5') ## upload mohamed_symptom.h5 and put the path here
def input_tokenizer(txt,tok):
  val=tok.texts_to_sequences(txt)
  p =pad_sequences(val,maxlen=43,padding='post',truncating='post')
  return p
def prediction_symptom(txt,classes,model):
  output=model.predict(txt)
  pred = np.argmax(output,axis=1)
  output_pred=classes[pred[0]]
  return output_pred

In [ ]:
while True:
  txt=input('Enter what is your feeling:')
  p=input_tokenizer(txt,tokenizer)
  output=prediction_symptom(p,all_symp,model_symp)
  print(output)

#Heart Desease

<h1>Find model_heart.pkl
in heart folder to load it </h1>

In [15]:
with open('/content/model_heart.pkl','rb') as m: ## upload model_heart and put the path here
    model_heart =pickle.load(m)


Example to test from dataset

In [24]:
arr1 = np.array([[ 61.,   1.,   4., 160.,   0.,   1.,   1., 145.,   0.,   1.,   2.]]) ### OUTPUT = 1
arr2 = np.array([[43. , 0., 2., 150., 186., 0., 0., 154., 0., 0., 1.]]) ### OUTPUT = 0

In [26]:
pred=model_heart.predict(arr1)
pred

array([1])

**Run this function if you want to input yourself the data the sex input should be of type string (male or female)**

In [29]:
def input_pred_heart():
  age=float(input('age: '))
  sex=str(input('sex: '))
  chest_pain=float(input('chest pain: '))
  resting_bp_s=float(input('resting_bp_s: '))
  cholesterol=float(input('cholesterol: '))
  fasting_blood_sugar=float(input('fasting blood sugar: '))
  resting_ecg=float(input('resting ecg: '))
  max_heart_rate=float(input('max heart rate: '))
  exercise_angina=float(input('exercise angina: '))
  oldpeak=float(input('oldpeak: '))
  ST_slope=float(input('ST slope: '))
  if sex.lower=='male':
    sex=float(1)
  else:
    sex=float(0)
  data=np.array([age,sex,chest_pain,resting_bp_s,cholesterol,fasting_blood_sugar,resting_ecg,max_heart_rate,exercise_angina,oldpeak,ST_slope])
  data=np.expand_dims(data,axis=0)
  print(data.shape)
  pred = model_heart.predict(data)
  s="Can't predict"
  if pred[0]==1:
    s="You have heart diease"
  if pred[0]==0:
    s="You don't have heart diease"
  return s


In [31]:
output_heart_disease_model=input_pred_heart()
output_heart_disease_model

age: 65
sex: male
chest pain: 4
resting_bp_s: 140
cholesterol: 306
fasting blood sugar: 1
resting ecg: 0
max heart rate: 87
exercise angina: 1
oldpeak: 1.5
ST slope: 2
(1, 11)


'You have heart diease'

##ChatBot Model

<h1>Find intents.json , words.pkl , classes.pkl , chatbotmodel.h5
in Chatbot folder to load them </h1>

In [34]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("/content/intents.json").read()) ## upload intents.json and put the path here
words = pickle.load(open('/content/words.pkl', 'rb')) ## upload words.pkl and put the path here
classes = pickle.load(open('/content/classes.pkl', 'rb')) ## upload classes.pkl and put the path here
model_chatbot = tf.keras.models.load_model('/content/chatbotmodel.h5') ## upload chatbotmodel.h5 and put the path here

def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower())
                      for word in sentence_words]
    return sentence_words
def bagw(sentence):

	# separate out words from the input sentence
	sentence_words = clean_up_sentences(sentence)
	bag = [0]*len(words)
	for w in sentence_words:
		for i, word in enumerate(words):

			# check whether the word
			# is present in the input as well
			if word == w:

				# as the list of words
				# created earlier.
				bag[i] = 1
	# return a numpy array
	return np.array(bag)
def predict_class(sentence,classes):
    bow = bagw(sentence)
    res = model_chatbot.predict(np.array([bow]))
    cl =np.argmax(res,axis=1)
    p = classes[cl[0]]
    return p

def get_response(txt,classes,intents):
    p =predict_class(txt,classes)


    for i in intents['intents']:
        if p==i['tag']:
            respond =i['responses']

    return (respond)

tags to ask about ['Cuts','Abrasions',
 'stings',
 'Splinter',
 'Sprains',
 'Strains',
 'Fever',
 'Nasal Congestion',
 'Cough',
 'Sore Throat',
 'Gastrointestinal problems',
 'Skin problems',
 'Abdonominal Pain',
 'Bruises',
 'Broken Toe',
 'Choking',
 'Wound',
 'Diarrhea',
 'Frost bite',
 'Heat Exhaustion',
 'Heat Stroke',
 'Insect Bites',
 'nose bleed',
 'Pulled Muscle',
 'Rectal bleeding',
 'Sun Burn',
 'Testicle Pain',
 'Vertigo',
 'Normal Bleeding',
 'Eye Injury',
 'Chemical Burn',
 'Poison',
 'Teeth',
 'seizure',
 'Head Injury',
 'Fainting',
 'Headache',
 'Cold',
 'Rash',
 'snake bite',
 'animal bite',
 'Drowning',
 'CPR',
 'Fracture']

 **Question Example:** What to do if someone is Drowning?,
 Do Abrasions cause scars?
 what to do if i get a strain?
 Which medicine to take if I got pulled muscle?

In [ ]:
while True:
  txt=input('chat to Me: ')
  res = get_response(txt,classes,intents)
  print(res[0]+'\n')

##Covid 19 Detector


<h1>Find detection_model.h5
in Covid 19 folder to load it </h1>

In [48]:
all_detect=['Covid', 'Normal', 'Viral Pneumonia']
model_detection = tf.keras.models.load_model('detection_model.h5') ## upload detection_model.h5 and put the path here
def predict_transfer_detect(img_path,classes,model):
  r=tf.keras.layers.Resizing(256,256)
  read_im=plt.imread(img_path)
  out=r(read_im)
  p=np.argmax(model.predict(np.expand_dims(out,axis=0)),axis=1)
  val=classes[p[0]]
  return val

In [49]:
im_path='/content/0120.jpg' ###upload an image and put the path here
output = predict_transfer_detect(im_path,all_detect,model_detection)
output

1/1 [==============================] - 2s 2s/step


'Covid'